In [0]:
%run "./startup" 

In [0]:
params = WidgetParams(
    {
        "sr_subproceso": str,
        "sr_folio": str
    }
)
params.validate()
query = QueryManager()
db = DBXConnectionManager()
queries_df = query.get_sql_list()
# filtar solo los registros donde la columan 'Archivo SQL' comiencen por el nombre del notebook
display(queries_df.filter(col("Archivo SQL").startswith("PRUEBA_COMMIT")))

In [0]:
statement_001 = query.get_statement(
"PRUEBA_COMMIT.sql"
)

#Ejecuta la Sentencia de arriba
execution = db.execute_oci_dml(
    statement=statement_001, async_mode=False
)

In [0]:
%sql
SELECT
*
FROM dbx_mit_dev_1udbvf_workspace.default.DELTA_0400_ARCH_RECH_SUM_AG_365_202507221150382822

In [0]:
%sql
SELECT
substr((FTN_NUM_CTA_INVDUAL:: STRING),1,charindex('.',(FTN_NUM_CTA_INVDUAL:: STRING))-1) AS FTN_NUM_CTA_INVDUAL,
FTC_FOLIO_BITACORA,
CASE WHEN FTN_SDO_AIVS IS NULL THEN 0 ELSE CAST(FTN_SDO_AIVS AS DECIMAL(15,2)) END AS FTN_SDO_AIVS,
CASE WHEN FTN_MONTO_PESOS IS NULL THEN 0 ELSE CAST(FTN_MONTO_PESOS AS DECIMAL(15,2)) END AS FTN_MONTO_PESOS,
FTN_MONTO_PESOS AS FTN_MONTO_PESOS_2,
1 AS REG
FROM --#DELTA_SUMARIO_SOLMARC#
dbx_mit_dev_1udbvf_workspace.default.delta_0399_arc_acep_lin2_tf_200_campos_202507171410392809

In [0]:
# Tamaño de un archivo individual
file_info = dbutils.fs.ls("abfss://nci-repository@datalakedev1udbvf.dfs.core.windows.net/PROCESAR/")
# dbutils.fs.ls devuelve lista; si es ruta directa al archivo, listará 1 elemento
for f in files:
   print(f.name)

   print(f.size)   

In [0]:
from pyspark.sql.functions import col, upper
from pyspark.sql.types import StructType, StructField, StringType, LongType

# 1. Definir la ruta del archivo/directorio
file_path = "abfss://nci-repository@datalakedev1udbvf.dfs.core.windows.net/PROCESAR/" # Reemplaza con tu ruta real

# Usar dbutils.fs.ls para obtener los metadatos.
# Si es un solo archivo, la lista contendrá un solo elemento.
# Nota: dbutils.fs.ls() devuelve objetos FileInfo con atributos como 'path', 'name', 'size', 'isDir', etc.
files_list = dbutils.fs.ls(file_path)

   #display(files_list)


# Seleccionar solo las columnas 'nombre' y 'edad'
df_tamaño_arch = spark.createDataFrame(files_list).select("name", "size")

from pyspark.sql.functions import col, upper

# Seleccionar columnas, renombrarlas y aplicar funciones
df_tamaño_arch = df_tamaño_arch.select(
    upper(col("name")).alias("FTC_NOMBRE"),  # Renombrar la columna
    upper(col("size")).alias("FTN_TAM_ARCHIVO")) # Aplicar función (mayúsculas)

display(df_tamaño_arch)

In [0]:
# List contents of a folder
files = dbutils.fs.ls("abfss://nci-repository@datalakedev1udbvf.dfs.core.windows.net/PROCESAR/")
for f in files:
    print(f"Nombre del Archivo:", f.name, f.size)


#dbutils.fs.cp('abfss://nci-repository@datalakedev1udbvf.dfs.core.windows.net/PROCESAR/PTCFT_20251029_TRANSAG_ACEP.txt',
 #             "dbfs:/C:\\Users\\Profuturo\\Documents\\2025\\ATC\\TRASPASOS\\Evidencias_Gen_Arch_Resp\\PTCFT_20251029_TRANSAG_ACEP.txt")


In [0]:
# Preview file contents (first N bytes)
print(dbutils.fs.head("abfss://nci-repository@datalakedev1udbvf.dfs.core.windows.net/PROCESAR/PREFTD_20251121_76437_NOTDEV.txt", 190000000))



In [0]:
%sql
SELECT * FROM dbx_mit_dev_1udbvf_workspace.default.DELTA_0200_INS_ARCHIVO_ACEPTADOS_354_202511051844286616

In [0]:

    df = db.read_data("default", statement_001)  # Ensure 'statement_001' has balanced parentheses and correct syntax.

    display(df)
    display(str(df.count()) + " Registros Totales")

In [0]:
#GENERA UN DATAFRAME A PARTIR DE LA TABLA DELTA CREADA UN PASO ANTERIOR
df_archivo = db.read_delta(f"DELTA_0200_INS_ARCHIVO_ACEPTADOS_{params.sr_subproceso}_{params.sr_folio}")\
.select("FTN_ID_ARCHIVO",
        "FTC_NOMBRE",
        #"FTN_TAM_ARCHIVO",
        "FCN_ID_PROCESO",
        "FCN_ID_SUBPROCESO",
        "FTD_FEH_GEN",
        "FTD_FEH_ACT",
        "FTC_USU_ACT",
        "FTD_FEH_ARCHIVO",
        "FCN_ID_ESTATUS_ARCHIVO")

In [0]:
%sql
WITH 
RES_02 
AS(
 SELECT
  COUNT(1) ID_02
  ,SUM(TRY_CAST(substr(FTC_LINEA,475,15) AS BIGINT)) ID_03
  ,SUM(TRY_CAST(substr(FTC_LINEA,550,15) AS BIGINT)) ID_04
 FROM dbx_mit_dev_1udbvf_workspace.default.DELTA_VALIDA_CONTENIDO_76387
 WHERE substr(FTC_LINEA,1,2) = '02'
)
SELECT
 FTN_NO_LINEA
 ,CASE WHEN substr(FTC_LINEA,1,2) = '09' then NULL else substr(FTC_LINEA,1,2) end Tipo_de_registro
 ,CASE WHEN CAST(substr(FTC_LINEA, 3, 9) AS BIGINT) = RES_02.ID_02 then NULL else substr(FTC_LINEA,3,9) end Cantidad_registros_detalle
 --,Filler
 ,CASE WHEN CAST(substr(FTC_LINEA, 42, 18) AS BIGINT) = RES_02.ID_03 then NULL else substr(FTC_LINEA,42,18) end Suma_numero_Aplicaciones_viv_97
 ,CASE WHEN CAST(substr(FTC_LINEA, 60, 18) AS BIGINT) = RES_02.ID_04 then NULL else substr(FTC_LINEA,60,18) end Suma_saldo_Viv_97
 --,CASE WHEN substr(FTC_LINEA,72,659) = REPEAT(' ', 659) then NULL WHEN LENGTH(TRIM(substr(FTC_LINEA,72,659))) = 0 then 'Registro_Vacio' else substr(FTC_LINEA,72,659) end Filler
FROM dbx_mit_dev_1udbvf_workspace.default.DELTA_VALIDA_CONTENIDO_76387
 JOIN RES_02 ON 1 = 1
WHERE substr(FTC_LINEA,1,2) = '09'

In [0]:
%sql
WITH RES_02 AS(
 SELECT
  COUNT(1) ID_02
  ,SUM(TRY_CAST(substr(FTC_LINEA,66,15) AS BIGINT)) ID_03
  ,SUM(TRY_CAST(substr(FTC_LINEA,101,15) AS BIGINT)) ID_04
  ,SUM(TRY_CAST(substr(FTC_LINEA,118,15) AS BIGINT)) ID_05
  ,SUM(TRY_CAST(substr(FTC_LINEA,153,15) AS BIGINT)) ID_06
 FROM dbx_mit_dev_1udbvf_workspace.default.DELTA_VALIDA_CONTENIDO_76411
 WHERE substr(FTC_LINEA,1,2) = '02'
)
SELECT
 FTN_NO_LINEA
 ,CASE WHEN substr(FTC_LINEA,1,2) = '09' then NULL else substr(FTC_LINEA,1,2) end Tipo_de_registro
 ,CASE WHEN CAST(substr(FTC_LINEA, 3, 9) AS BIGINT) = RES_02.ID_02 then NULL else substr(FTC_LINEA,3,9) end Cantidad_registros_detalle
 ,CASE WHEN CAST(substr(FTC_LINEA, 12, 18) AS BIGINT) = RES_02.ID_03 then NULL else substr(FTC_LINEA,12,18) end Aplicacion_Intereses_Viv_FOVISSSTE_92
 ,CASE WHEN CAST(substr(FTC_LINEA, 30, 15) AS BIGINT) = RES_02.ID_04 then NULL else substr(FTC_LINEA,30,15) end Importe_Fondo_viv_FOVISSSTE_92
 ,CASE WHEN CAST(substr(FTC_LINEA, 45, 18) AS BIGINT) = RES_02.ID_05 then NULL else substr(FTC_LINEA,45,18) end Aplicacion_Intereses_Viv_FOVISSSTE_2008
 ,CASE WHEN CAST(substr(FTC_LINEA, 63, 15) AS BIGINT) = RES_02.ID_06 then NULL else substr(FTC_LINEA,63,15) end Importe_Fondo_viv_FOVISSSTE_2008
 --,Codigo_Resultado_de_la_Operacion
 --,Diagnostico_del_Proceso
 --,Filler
FROM dbx_mit_dev_1udbvf_workspace.default.DELTA_VALIDA_CONTENIDO_76411
 JOIN RES_02 ON 1 = 1
WHERE substr(FTC_LINEA,1,2) = '09'

In [0]:
%sql
 SELECT
  COUNT(1) ID_02
  ,SUM(TRY_CAST(substr(FTC_LINEA,66,15) AS BIGINT)) ID_03
  ,SUM(TRY_CAST(substr(FTC_LINEA,101,15) AS BIGINT)) ID_04
  ,SUM(TRY_CAST(substr(FTC_LINEA,118,15) AS BIGINT)) ID_05
  ,SUM(TRY_CAST(substr(FTC_LINEA,153,15) AS BIGINT)) ID_06

    TRY_CAST(substr(FTC_LINEA,475,9)||'.'||substr(FTC_LINEA,483,6) AS DECIMAL(15,6)) ID_03
  ,TRY_CAST(substr(FTC_LINEA,490,13)||'.'||substr(FTC_LINEA,503,2) AS DECIMAL(15,2)) ID_04
 FROM dbx_mit_dev_1udbvf_workspace.default.DELTA_VALIDA_CONTENIDO_76411
 WHERE substr(FTC_LINEA,1,2) = '02'


In [0]:
%sql
SELECT * FROM dbx_mit_dev_1udbvf_workspace.default.DELTA_VALIDA_CONTENIDO_76366

In [0]:
%sql
 SELECT
  --COUNT(1) ID_02
  --substr(FTC_LINEA,475,9)
   TRY_CAST(substr(FTC_LINEA,475,9)||'.'||substr(FTC_LINEA,483,6) AS DECIMAL(18,6)) ID_03
  ,TRY_CAST(substr(FTC_LINEA,490,13)||'.'||substr(FTC_LINEA,503,2) AS DECIMAL(15,2)) ID_04
  --substr(FTC_LINEA,475,13)||'.'||substr(FTC_LINEA,488,2) AS ID_03,
  --substr(FTC_LINEA,490,13)||'.'||substr(FTC_LINEA,503,2) AS  ID_04
 FROM dbx_mit_dev_1udbvf_workspace.default.DELTA_VALIDA_CONTENIDO_76308
 WHERE substr(FTC_LINEA,1,2) = '02'

In [0]:
# List contents of a folder
files = dbutils.fs.ls("abfss://nci-repository@datalakedev1udbvf.dfs.core.windows.net/PROCESAR/")
for f in files:
    print(f"Nombre del Archivo:", f.name, f.size)


#dbutils.fs.cp('abfss://nci-repository@datalakedev1udbvf.dfs.core.windows.net/PROCESAR/PTCFT_20251029_TRANSAG_ACEP.txt',
 #             "dbfs:/C:\\Users\\Profuturo\\Documents\\2025\\ATC\\TRASPASOS\\Evidencias_Gen_Arch_Resp\\PTCFT_20251029_TRANSAG_ACEP.txt")
